In [1]:
#default_exp proto_utils

# adapted from the torchmeta code
# TODO: use this in vanilla protonet code
import torch
import torch.nn.functional as F

In [3]:
#export
def euclidean_distance(x, y):
    """
    x, y have shapes (batch_size, num_examples, embedding_size).
    x is prototypes, y are embeddings in most cases
    """
    return torch.sum((x.unsqueeze(2) - y.unsqueeze(1))** 2, dim=-1)

In [4]:
#export

def cosine_similarity(x, y):
    """x, y have shapes (batch_size, num_examples, embedding_size)."""
    
    # compute dot prod similarity x_i.T y_i (numerator)
    dot_similarity = torch.bmm(x, y.permute(0, 2, 1))
    
    # compute l2 norms ||x_i|| * ||y_i||
    x_norm = x.norm(p=2, dim=-1, keepdim=True)
    y_norm = y.norm(p=2, dim=-1, keepdim=True)
    
    norms = torch.bmm(x, y.permute(0, 2, 1)) + 1e-8
    
    return dot_similarity / norms
    

In [5]:
#export

def get_num_samples(targets, num_classes, dtype=None):
    batch_size = targets.size(0)
    with torch.no_grad():
        ones = torch.ones_like(targets, dtype=dtype)
        num_samples = ones.new_zeros((batch_size, num_classes))
        num_samples.scatter_add_(1, targets, ones)
    return num_samples

In [7]:
#export

def get_prototypes(self, emb, targets, num_classes):
        """Compute the prototypes (the mean vector of the embedded training/support 
        points belonging to its class) for each classes in the task.
        Parameters
        ----------
        embeddings : `torch.FloatTensor` instance
            A tensor containing the embeddings of the support points. This tensor 
            has shape `(batch_size, num_examples, embedding_size)`.
        targets : `torch.LongTensor` instance
            A tensor containing the targets of the support points. This tensor has 
            shape `(batch_size, num_examples)`.
        num_classes : int
            Number of classes in the task.
        Returns
        -------
        prototypes : `torch.FloatTensor` instance
            A tensor containing the prototypes for each class. This tensor has shape
            `(batch_size, num_classes, embedding_size)`.
        """

        batch_size, emb_size = emb.size(0), emb.size(-1)

        num_samples = self.get_num_samples(targets, num_classes, dtype=emb.dtype)
        num_samples.unsqueeze_(-1)
        num_samples = torch.max(num_samples, torch.ones_like(num_samples))

        prototypes = emb.new_zeros((batch_size, num_classes, emb_size))
        indices = targets.unsqueeze(-1).expand_as(emb)

        prototypes.scatter_add_(1, indices, emb).div_(num_samples)

        return prototypes

In [8]:
#export
def prototypical_loss(prototypes, embeddings, targets, 
                      distance='euclidean', **kwargs):
    """Compute the loss (i.e. negative log-likelihood) for the prototypical 
    network, on the test/query points.

    Parameters
    ----------
    prototypes : `torch.FloatTensor` instance
        A tensor containing the prototypes for each class. This tensor has shape 
        `(batch_size, num_classes, embedding_size)`.

    embeddings : `torch.FloatTensor` instance
        A tensor containing the embeddings of the query points. This tensor has 
        shape `(batch_size, num_examples, embedding_size)`.

    targets : `torch.LongTensor` instance
        A tensor containing the targets of the query points. This tensor has 
        shape `(batch_size, num_examples)`.

    distance : `String`
        The distance measure to be used: 'eucliden' or 'cosine'

    Returns
    -------
    loss : `torch.FloatTensor` instance
        The negative log-likelihood on the query points.
    accuracy : `torch.FloatTensor` instance
        Mean accuracy on the query points.
    """
    if distance == 'euclidean':
        squared_distances = euclidean_distance(prototypes, embeddings)
        loss = F.cross_entropy(-squared_distances, targets, **kwargs)
        _, predictions = torch.min(squared_distances, dim=1)
        accuracy = torch.mean(predictions.eq(targets).float())
    elif distance == 'cosine':
        cosine_similarities = cosine_similarity(prototypes, embeddings)
        loss = F.cross_entropy(cosine_similarities, targets, **kwargs)
        _, predictions = torch.max(cosine_similarities, dim=1)
        accuracy = torch.mean(predictions.eq(targets).float())
    else:
        raise ValueError('Distance must be "euclidean" or "cosine"')
    return loss, accuracy.item()

In [10]:
from nbdev.export import notebook2script; notebook2script()

Converted 01_nn_utils.ipynb.
Converted 01b_data_loaders_pl.ipynb.
Converted 01c_grad_utils.ipynb.
Converted 01d_proto_utils.ipynb.
Converted 02_maml_pl.ipynb.
Converted 02b_iMAML.ipynb.
Converted 03_protonet_pl.ipynb.
No export destination, ignored:
#export 
Converted 03b_ProtoCLR.ipynb.
Converted 04_cactus.ipynb.
Converted index.ipynb.
